**Notebook 03 — Construcción de memoria/few-shot (guías+gold) + lista de excluidos (anti-leakage)**

**Objetivo**

Este notebook construye la memoria few-shot que se usará en los prompts del agente, a partir de:
- Guías de anotación MVP (guidelines_MVP_defs.txt)
- Corpus gold anotado

**Qué hace exactamente este NB03**
- Lee la guía en formato TXT y extrae citas literales entre corchetes [...]
→ solo con fines de auditoría y alineación conceptual (no se usan directamente en el prompt).
- Carga el gold y construye un pool de spans reales para las 4 etiquetas MVP:
    - OBJETO
    - PRECIO_DEL_CONTRATO
    - DURACION_TOTAL_DEL_CONTRATO
    - RESOLUCION
- Selecciona un conjunto reducido y representativo de ejemplos (memoria), uno por criterio/subtipo cuando aplica.
- Genera:
    - outputs/memory/memory_examples.json
    - outputs/memory/blocked_doc_uids.json (para evitar leakage en val/test/prompt_regression en NB02)

**Importante**
- ❌ No se usa DOCX (antes las guías estaban en DOCX pero no cogía los corchetes)
- ❌ No se usa corpus sin anotar
- ✅ Toda la memoria sale del gold

En este NB03 NO necesitamos cargar el corpus sin anotar para construir memoria, porque la memoria la construimos desde el gold (como decidimos). Solo usaremos las guías para ver/confirmar criterios y ejemplos.

In [1]:
from pathlib import Path
ROOT = Path("/home/jovyan/inesagent")
assert ROOT.exists()
print("ROOT:", ROOT)


ROOT: /home/jovyan/inesagent


In [2]:
from pathlib import Path

ROOT = Path("/home/jovyan/inesagent")
PATH_GUIDE = ROOT / "config" / "guidelines_MVP_defs.txt"

print("PATH_GUIDE existe:", PATH_GUIDE.exists())
print(PATH_GUIDE)


PATH_GUIDE existe: True
/home/jovyan/inesagent/config/guidelines_MVP_defs.txt


In [3]:
#Imports y configuración base  
from pathlib import Path
import json, re, hashlib, random
from collections import defaultdict
from typing import List, Dict, Any


In [4]:
#Rutas del proyecto (servidor) 
ROOT = Path("/home/jovyan/inesagent")
assert ROOT.exists(), f"ROOT no existe: {ROOT}"
print("ROOT:", ROOT)

# Guía MVP en TXT (fuente de criterios / ejemplos)
PATH_GUIDE = ROOT / "config" / "guidelines_MVP_defs.txt"
assert PATH_GUIDE.exists(), f"No encuentro {PATH_GUIDE}"

# Gold anotado
PATH_GOLD = ROOT / "gold" / "corpus_annotated.jsonl"
assert PATH_GOLD.exists(), f"No encuentro {PATH_GOLD}"

# Salidas
OUT_DIR = ROOT / "outputs" / "memory"
OUT_DIR.mkdir(parents=True, exist_ok=True)

PATH_MEMORY = OUT_DIR / "memory_examples.json"
PATH_BLOCKED = OUT_DIR / "blocked_doc_uids.json"

SEED = 42
random.seed(SEED)



ROOT: /home/jovyan/inesagent


In [5]:
#configuracion MVP 
MVP_LABELS = {
    "P1": "OBJETO",
    "P2": "PRECIO_DEL_CONTRATO",
    "P4": "DURACION_TOTAL_DEL_CONTRATO",
    "P9": "RESOLUCION",
}



In [6]:
#Utilidades generales 
def stable_uid(text: str) -> str:
    return hashlib.sha1(text.encode("utf-8")).hexdigest()

def load_jsonl(path: Path) -> List[Dict[str, Any]]:
    rows = []
    with open(path, encoding="utf-8") as f:
        for line in f:
            if line.strip():
                rows.append(json.loads(line))
    return rows

def normalize(s: str) -> str:
    return " ".join((s or "").lower().split())


In [7]:
#leer guia TXT y extraer citas [...] Solo auditoría / trazabilidad, no afecta a la memoria final
text_guide = PATH_GUIDE.read_text(encoding="utf-8")

BRACKET_RE = re.compile(r"\[(.+?)\]", flags=re.DOTALL)

quotes = [q.strip() for q in BRACKET_RE.findall(text_guide) if q.strip()]

# deduplicar preservando orden
seen = set()
quotes_unique = []
for q in quotes:
    key = normalize(q)
    if key not in seen:
        seen.add(key)
        quotes_unique.append(q)

print("Citas únicas extraídas de la guía:", len(quotes_unique))
print("Ejemplos:")
for q in quotes_unique[:5]:
    print("-", q[:200])



Citas únicas extraídas de la guía: 28
Ejemplos:
- suministro de armarios consigna con apertura individual mediante cerradura electrónica para los edificios del Banco de España en Lumbreras
- Explotación de la aplicación móvil para personal de la Armada Española del expediente 2022/EA02/00001693E
- SERVICIOS DE RESTAURACIÓN DE UN CONJUNTO DE 45 OBRAS PERTENECIENTES AL FONDO HISTÓRICO DE LA BIBLIOTECA DEL SENADO (LOTE 1)
- SERVICIO         DE         CARGA         Y         DESCARGA         DE    ELEMENTOS  ESCENOGRÁFICOS Y DELA CARROZA PARA LA FUNDACIÓN DEL  TEATRO REAL F.S.P.
- 14.567 €


In [8]:
#Cargar gold y preparar indices 
gold = load_jsonl(PATH_GOLD)
print("Docs gold:", len(gold))

# En gold, la clave canónica es 'id'. Solo si faltase (caso raro) usamos sha1(texto).
gold_id_set = set()
for d in gold:
    if "id" in d:
        gold_id_set.add(d["id"])

key_to_text = {}
key_to_tags = {}
key_type = {}  # key -> 'id' o 'uid'

for d in gold:
    txt = d.get("text", "")
    if not txt:
        continue
    if "id" in d:
        k = d["id"]
        key_type[k] = "id"
    else:
        k = stable_uid(txt)
        key_type[k] = "uid"
    key_to_text[k] = txt
    key_to_tags[k] = d.get("tags", [])


Docs gold: 373


Esta salida quiere decir:
- _**“Docs con al menos 1 span MVP: 373”**_:  En corpus_annotated.jsonl, hay 373 documentos que tienen al menos una anotación de OBJETO / PRECIO_DEL_CONTRATO / DURACION_TOTAL_DEL_CONTRATO / RESOLUCION.



In [9]:
#Pool de spans MVP desde gold 
pool = defaultdict(list)

for k, txt in key_to_text.items():
    for t in key_to_tags.get(k, []):
        lab = t.get("tag")
        if lab not in MVP_LABELS.values():
            continue
        s, e = int(t["start"]), int(t["end"])
        if 0 <= s < e <= len(txt):
            span_txt = txt[s:e].strip()
            if span_txt:
                pool[lab].append({
                    # por compatibilidad, mantenemos el nombre 'doc_uid',
                    # pero aquí es el 'id' de gold (o uid si el doc no tenía id)
                    "doc_uid": k,
                    "key_type": key_type.get(k, "id"),
                    "start": s,
                    "end": e,
                    "text": span_txt
                })

for lab in MVP_LABELS.values():
    print(lab, "spans:", len(pool[lab]))


OBJETO spans: 925
PRECIO_DEL_CONTRATO spans: 436
DURACION_TOTAL_DEL_CONTRATO spans: 468
RESOLUCION spans: 526


In [10]:
#Seleccion de memoria (simple, robusta) - estrategia: P1,P2 (1 ej), P4,P9 (4 ejs variados) 
memory = []
blocked_uids = set()

def add_example(label, criterion, ex):
    memory.append({
        "label": label,
        "criterion": criterion,
        "doc_uid": ex["doc_uid"],
        "key_type": ex.get("key_type","id"),
        "start": ex["start"],
        "end": ex["end"]
    })
    blocked_uids.add(ex["doc_uid"])


In [11]:
#P1 y P2 (1 ejemplo de cada una, ya que tienen su criterio definido) 
for code in ["P1", "P2"]:
    label = MVP_LABELS[code]
    ex = random.choice(pool[label])
    add_example(
        label=label,
        criterion=f"{code} — ejemplo representativo (gold)",
        ex=ex
    )


In [12]:
#P4 y P9 (variedad de ejemplos) 
def pick_k(label, k):
    xs = pool[label].copy()
    random.shuffle(xs)
    return xs[:k]

for code, k in [("P4", 4), ("P9", 4)]:
    label = MVP_LABELS[code]
    for ex in pick_k(label, k):
        add_example(
            label=label,
            criterion=f"{code} — ejemplo representativo (gold)",
            ex=ex
        )


In [13]:
#Validacion y guardado 
print("Memory examples:", len(memory))

# Separar bloqueos por tipo de clave (id vs uid)
blocked_ids = sorted({k for k in blocked_uids if isinstance(k, int) or (isinstance(k,str) and k in gold_id_set)})
blocked_uids_only = sorted({k for k in blocked_uids if k not in set(blocked_ids)})

print("Docs bloqueados (id):", len(blocked_ids))
print("Docs bloqueados (uid):", len(blocked_uids_only))

# 1) Memoria (ejemplos)
PATH_MEMORY.write_text(
    json.dumps(memory, ensure_ascii=False, indent=2),
    encoding="utf-8"
)

# 2) Fichero nuevo robusto (recomendado)
PATH_BLOCKED_KEYS = OUT_DIR / "blocked_keys_by_memory.json"
PATH_BLOCKED_KEYS.write_text(
    json.dumps({"blocked_ids": blocked_ids, "blocked_uids": blocked_uids_only}, ensure_ascii=False, indent=2),
    encoding="utf-8"
)

# 3) Fichero legacy (por compatibilidad con notebooks antiguos)
PATH_BLOCKED.write_text(
    json.dumps(sorted(blocked_uids), ensure_ascii=False, indent=2),
    encoding="utf-8"
)

print("Guardado:")
print("-", PATH_MEMORY)
print("-", PATH_BLOCKED_KEYS)
print("-", PATH_BLOCKED, "(legacy)")


Memory examples: 10
Docs bloqueados (id): 10
Docs bloqueados (uid): 0
Guardado:
- /home/jovyan/inesagent/outputs/memory/memory_examples.json
- /home/jovyan/inesagent/outputs/memory/blocked_keys_by_memory.json
- /home/jovyan/inesagent/outputs/memory/blocked_doc_uids.json (legacy)


**La guía TXT tiene dos patrones de subtipo:**
- P1.1 <título en la misma línea> ✅
- P4.01 sin título (solo el código) y el contenido va en la línea siguiente ✅
- y además hay P4.09 (con 0 delante o sin 0): ambos deben entrar.

RE_SUB actual exige “algo” después del código (porque captura (.+?)), así que se salta los casos P4.01 que vienen solos.

Corregimos regex + parser que:
- Detecta P1/P2/P4/P9 main (sin confundir P1.1 con P1.)
- Detecta subtipos con y sin título en la misma línea (P4.01 solo)
- Soporta P4.9 y P4.09 (uno o dos dígitos)
- Captura Criterio: tanto para main como para subtipo
- Captura ejemplos [...] incluso si el bloque de ejemplos ocupa varias líneas
- Al final imprime lo que pedías: tipos detectados + subtipos por tipo + nº de citas por tipo/subtipo

Requisito previo: tener text_guide con el contenido del TXT (ya lo tienes al leer guidelines_MVP_defs.txt). 

**Por qué esta versión sí captura tus casos raros**
- P4.01: ahora RE_SUB acepta que el título sea vacío ((.*)), por tanto lo detecta como subtipo aunque no haya texto en esa línea.
- P4.9 y P4.09: \d{1,2} admite 1 o 2 dígitos.
- No confunde P1.1 con P1.: RE_MAIN tiene (?!\d).

In [14]:
#parsear guia TXT en estructura (tipos/subtipos+criterio) 
import re
from collections import defaultdict

# MAIN: P1. Objeto  (pero NO P1.1 ...)
RE_MAIN = re.compile(r"^\s*(P(?:1|2|4|9))\.(?!\d)\s*(.+?)\s*$", re.IGNORECASE)

# SUBTIPO:
# - P1.1 Título...
# - P4.01
# - P4.9
# - P4.09
# Permitimos título opcional (puede estar vacío)
RE_SUB = re.compile(r"^\s*(P(?:1|2|4|9)\.(?:\d{1,2})(?:\.\d+)*)\s*(.*)\s*$", re.IGNORECASE)

RE_CRIT = re.compile(r"^\s*Criterio\s*:\s*(.+?)\s*$", re.IGNORECASE)
RE_EJ   = re.compile(r"^\s*Ejemplos?\s*:\s*(.*)\s*$", re.IGNORECASE)

BRACKET_RE = re.compile(r"\[(.+?)\]", flags=re.DOTALL)

def norm_line(s: str) -> str:
    return (s or "").replace("\ufeff", "").strip()

guide = {
    "P1": {"title": "", "criterion": "", "subtypes": {}},
    "P2": {"title": "", "criterion": "", "subtypes": {}},
    "P4": {"title": "", "criterion": "", "subtypes": {}},
    "P9": {"title": "", "criterion": "", "subtypes": {}},
}

def ensure_sub(main: str, subcode: str):
    guide[main]["subtypes"].setdefault(subcode, {"title": "", "criterion": "", "examples": []})

lines = [norm_line(ln) for ln in text_guide.splitlines()]
lines = [ln for ln in lines if ln]  # remove empty

current_code = None     # "P1" o "P4.01" etc.
current_main = None     # "P1" etc.
in_examples_block = False

for ln in lines:
    # 1) MAIN
    m = RE_MAIN.match(ln)
    if m:
        main = m.group(1).upper()
        title = m.group(2).strip()
        if main in guide:
            guide[main]["title"] = title
            current_main = main
            current_code = main
            in_examples_block = False
        continue

    # 2) SUBTIPO (título puede estar vacío)
    m = RE_SUB.match(ln)
    if m:
        subcode = m.group(1).upper()
        title = (m.group(2) or "").strip()
        main = subcode.split(".")[0]
        if main in guide:
            ensure_sub(main, subcode)
            # Si el título va en la misma línea, lo guardamos; si no, se queda vacío
            if title:
                guide[main]["subtypes"][subcode]["title"] = title
            current_main = main
            current_code = subcode
            in_examples_block = False
        continue

    # 3) CRITERIO (se asigna al bloque actual: subtipo si estamos en subtipo, si no main)
    m = RE_CRIT.match(ln)
    if m and current_code:
        crit = m.group(1).strip()
        if "." in current_code:
            main = current_code.split(".")[0]
            ensure_sub(main, current_code)
            guide[main]["subtypes"][current_code]["criterion"] = crit
        else:
            guide[current_code]["criterion"] = crit
        in_examples_block = False
        continue

    # 4) EJEMPLOS: activa modo ejemplos; puede haber texto después de "Ejemplos:" en la misma línea
    m = RE_EJ.match(ln)
    if m:
        in_examples_block = True
        ln = (m.group(1) or "").strip()
        # seguimos a extracción de corchetes debajo (no continue)

    # 5) Extraer quotes si estamos en bloque de ejemplos o si la línea contiene corchetes
    if in_examples_block or ("[" in ln and "]" in ln):
        for mm in BRACKET_RE.finditer(ln):
            q = (mm.group(1) or "").strip()
            if not q:
                continue

            if current_code and "." in current_code:
                main = current_code.split(".")[0]
                ensure_sub(main, current_code)
                guide[main]["subtypes"][current_code]["examples"].append(q)

            elif current_code in guide:
                # ejemplo a nivel etiqueta main -> pseudo-subtipo MAIN
                pseudo = f"{current_code}.MAIN"
                ensure_sub(current_code, pseudo)
                if not guide[current_code]["subtypes"][pseudo]["title"]:
                    guide[current_code]["subtypes"][pseudo]["title"] = "(ejemplo a nivel etiqueta)"
                # criterio del main si existe
                if guide[current_code]["criterion"] and not guide[current_code]["subtypes"][pseudo]["criterion"]:
                    guide[current_code]["subtypes"][pseudo]["criterion"] = guide[current_code]["criterion"]
                guide[current_code]["subtypes"][pseudo]["examples"].append(q)

# ---- Report solicitado
print("Tipos detectados:", [k for k in guide if guide[k]["title"]])
for k in ["P1","P2","P4","P9"]:
    subs = guide[k]["subtypes"]
    # recuento subtipos "reales" (sin contar P1.MAIN / P2.MAIN pseudo)
    real = [s for s in subs if not s.endswith(".MAIN")]
    print(f"{k} subtipos (incluyendo códigos tipo P4.01): {len(real)} | +MAIN_pseudo: {len(subs)-len(real)}")

# opcional: ver rápidamente qué subtipos detectó
for k in ["P1","P2","P4","P9"]:
    real = sorted([s for s in guide[k]["subtypes"] if not s.endswith(".MAIN")])
    print(f"\n{k} subtipos detectados:", real[:30], ("..." if len(real)>30 else ""))




Tipos detectados: ['P1', 'P2', 'P4', 'P9']
P1 subtipos (incluyendo códigos tipo P4.01): 3 | +MAIN_pseudo: 1
P2 subtipos (incluyendo códigos tipo P4.01): 3 | +MAIN_pseudo: 1
P4 subtipos (incluyendo códigos tipo P4.01): 9 | +MAIN_pseudo: 0
P9 subtipos (incluyendo códigos tipo P4.01): 12 | +MAIN_pseudo: 0

P1 subtipos detectados: ['P1.1', 'P1.2', 'P1.3'] 

P2 subtipos detectados: ['P2.1', 'P2.2', 'P2.3'] 

P4 subtipos detectados: ['P4.01', 'P4.02', 'P4.03', 'P4.04', 'P4.05', 'P4.06', 'P4.07', 'P4.08', 'P4.09'] 

P9 subtipos detectados: ['P9.01', 'P9.02', 'P9.03', 'P9.04', 'P9.05', 'P9.06', 'P9.07', 'P9.08', 'P9.09', 'P9.10', 'P9.11', 'P9.12'] 


 **¿Por qué no hay MAIN de P9?**
 En el TXT, de P9 no hay ejemplos “a nivel main” (P9 sin decimal) con corchetes [...], o al menos no los estás capturando bajo el bloque current_code == "P9".
- En tu parser, el “MAIN_pseudo” solo se crea cuando ocurre esto:
- Estamos dentro de un bloque main (por ejemplo current_code == "P9")
- Aparece una cita [...] en líneas de “Ejemplos” entonces guardamos esa cita como P9.MAIN (pseudo-subtipo)

En tu extracto de P9, justo después de:

`P9. Resolución
Criterio: ...
P9.01
Ejemplos: [...]
P9.02
Ejemplos: [...]
...`


Todas las citas [...] aparecen bajo P9.01, P9.02, etc. No hay una sección tipo:
`P9. Resolución`
`Ejemplos: [ ... ]` antes de P9.01.
- Así que el parser nunca necesita crear P9.MAIN: ya está asignando los ejemplos al subtipo concreto, y por eso te sale:
- P9 real subtypes = 12
- +MAIN_pseudo = 0

Mientras que en P1 y P2 sí estás viendo MAIN_pseudo: 1 porque en el TXT sí hay un ejemplo general en el bloque P1. y P2.:
- P1: Ejemplos: ... [suministro de armarios ...] antes de P1.1
- P2: Ejemplos: ... [14.567 €] antes de P2.1
- En P4 te sale MAIN_pseudo: 1 por otra razón interesante: tu P4 “tiene Ejemplos” antes del primer subtipo o te está entrando algún [...] cuando current_code == "P4", probablemente por el bloque inicial de P4 si en el TXT hay alguna cita entre corchetes en la sección general (o si alguna línea con corchetes aparece antes de detectar P4.01).

In [15]:
#creamos memory_selected_AUTO.json (aleatorio) (MVP 14-16) con convención P4/P9 fija 
import random
random.seed(SEED)

FIXED_CRITERION = {
    "P4": "P4 – criterio único (subtipos solo ejemplificativos)",
    "P9": "P9 – criterio único (subtipos solo ejemplificativos)",
}

P_TO_LABEL = {
    "P1": "OBJETO",
    "P2": "PRECIO_DEL_CONTRATO",
    "P4": "DURACION_TOTAL_DEL_CONTRATO",
    "P9": "RESOLUCION",
}

def pick_one(label):
    xs = pool.get(label, [])
    return random.choice(xs) if xs else None

memory_selected_AUTO = []

# P1: 1 MAIN + 1 por subtipo real (los que existan)
p1_subs = [s for s in guide["P1"]["subtypes"].keys() if s != "P1.MAIN"]
# Asegura MAIN
p1_codes = ["P1.MAIN"] + sorted([s for s in p1_subs if s.startswith("P1.")])[:3]  # si hay 3 subtipos en tu guía
for code in p1_codes:
    ex = pick_one(P_TO_LABEL["P1"])
    if not ex: 
        continue
    info = guide["P1"]["subtypes"].get(code, {"title":"", "criterion":""})
    criterion = f"{code} {info.get('title','')}".strip()
    if info.get("criterion"):
        criterion += f" — {info['criterion']}"
    memory_selected_AUTO.append({"label": P_TO_LABEL["P1"], "criterion": criterion, **{k: ex[k] for k in ["doc_uid","start","end"]}})

# P2: 1 MAIN + 1 por subtipo real
p2_subs = [s for s in guide["P2"]["subtypes"].keys() if s != "P2.MAIN"]
p2_codes = ["P2.MAIN"] + sorted([s for s in p2_subs if s.startswith("P2.")])[:3]
for code in p2_codes:
    ex = pick_one(P_TO_LABEL["P2"])
    if not ex: 
        continue
    info = guide["P2"]["subtypes"].get(code, {"title":"", "criterion":""})
    criterion = f"{code} {info.get('title','')}".strip()
    if info.get("criterion"):
        criterion += f" — {info['criterion']}"
    memory_selected_AUTO.append({"label": P_TO_LABEL["P2"], "criterion": criterion, **{k: ex[k] for k in ["doc_uid","start","end"]}})

# P4/P9: 4 ejemplos de variedad (por ahora random, se curará después)
for _ in range(4):
    ex = pick_one(P_TO_LABEL["P4"])
    if ex:
        memory_selected_AUTO.append({"label": P_TO_LABEL["P4"], "criterion": FIXED_CRITERION["P4"], **{k: ex[k] for k in ["doc_uid","start","end"]}})
for _ in range(4):
    ex = pick_one(P_TO_LABEL["P9"])
    if ex:
        memory_selected_AUTO.append({"label": P_TO_LABEL["P9"], "criterion": FIXED_CRITERION["P9"], **{k: ex[k] for k in ["doc_uid","start","end"]}})

print("memory_selected_AUTO tamaño:", len(memory_selected_AUTO))
PATH_AUTO = OUT_DIR / "memory_selected_AUTO.json"
PATH_AUTO.write_text(json.dumps(memory_selected_AUTO, ensure_ascii=False, indent=2), encoding="utf-8")
print("Guardado:", PATH_AUTO)


memory_selected_AUTO tamaño: 16
Guardado: /home/jovyan/inesagent/outputs/memory/memory_selected_AUTO.json


In [16]:
#Este es el anti-leakage real: bloquea los docs usados en la memoria
blocked_keys = sorted({ex["doc_uid"] for ex in memory_selected_AUTO})

blocked_ids = sorted({k for k in blocked_keys if isinstance(k, int) or (isinstance(k,str) and k in gold_id_set)})
blocked_uids_only = sorted({k for k in blocked_keys if k not in set(blocked_ids)})

# Nuevo formato robusto
PATH_BLOCK = OUT_DIR / "blocked_keys_by_memory.json"
PATH_BLOCK.write_text(
    json.dumps({"blocked_ids": blocked_ids, "blocked_uids": blocked_uids_only}, ensure_ascii=False, indent=2),
    encoding="utf-8"
)

# Legacy para compatibilidad
PATH_BLOCK_LEGACY = OUT_DIR / "blocked_doc_uids_by_memory.json"
PATH_BLOCK_LEGACY.write_text(json.dumps(blocked_keys, ensure_ascii=False, indent=2), encoding="utf-8")

print("Docs bloqueados por memoria | ids:", len(blocked_ids), "| uids:", len(blocked_uids_only))
print("Guardado:", PATH_BLOCK)
print("Guardado:", PATH_BLOCK_LEGACY, "(legacy)")


Docs bloqueados por memoria | ids: 16 | uids: 0
Guardado: /home/jovyan/inesagent/outputs/memory/blocked_keys_by_memory.json
Guardado: /home/jovyan/inesagent/outputs/memory/blocked_doc_uids_by_memory.json (legacy)


**Construir pool de spans (solo gold_mvp, excluyendo removed)**

OJO: aquí hay dos modos:
- Modo A (recomendado): memoria NO debe venir de removed → excluimos removed del pool.
- Modo B: permites memoria desde removed pero luego bloqueas evaluación.

Como ya dijimos “retiramos esos documentos para evitar leakage”, vamos con Modo A.

In [17]:
#curated memory P1/P2 por subtipo con criterio, P4/P9 por variedad 
P_TO_LABEL = {
    "P1": "OBJETO",
    "P2": "PRECIO_DEL_CONTRATO",
    "P4": "DURACION_TOTAL_DEL_CONTRATO",
    "P9": "RESOLUCION",
}

FIXED_CRITERION = {
    "P4": "P4 – criterio único (subtipos solo ejemplificativos)",
    "P9": "P9 – criterio único (subtipos solo ejemplificativos)",
}

def fallback_random(label: str):
    xs = pool.get(label, [])
    return random.choice(xs) if xs else None

memory_selected = []
used_doc_uids = set()

def add_example(label, criterion, ex):
    memory_selected.append({
        "label": label,
        "criterion": criterion,
        "doc_uid": ex["doc_uid"],
        "start": ex["start"],
        "end": ex["end"],
    })
    used_doc_uids.add(ex["doc_uid"])

# --- P1/P2: 1 ejemplo por subtipo (si tiene criterio; si no, igual 1 por subtipo si hay)
for main in ["P1", "P2"]:
    label = P_TO_LABEL[main]
    for subcode, info in guide[main]["subtypes"].items():
        title = (info.get("title") or "").strip()
        crit  = (info.get("criterion") or "").strip()
        examples = info.get("examples", [])

        criterion_str = f"{subcode} {title}".strip()
        if crit:
            criterion_str += f" — {crit}"

        # No usamos quotes para localizar offsets (porque ya evitamos removed).
        # En vez de eso: sample directo del pool para ese label.
        fb = fallback_random(label)
        if fb:
            add_example(label, criterion_str + " (gold)", fb)

# --- P4/P9: variedad formal (sin subtipo conceptual)
dur_patterns = [
    ("formalizacion", re.compile(r"\bformalizaci[oó]n\b", re.IGNORECASE)),
    ("fechas", re.compile(r"\b\d{1,2}\s+de\s+[a-záéíóú]+\s+de\s+\d{4}\b", re.IGNORECASE)),
    ("anos", re.compile(r"\b\d+\s+a[nñ]os?\b", re.IGNORECASE)),
    ("meses", re.compile(r"\b\d+\s+mes(?:es)?\b", re.IGNORECASE)),
    ("dias", re.compile(r"\b\d+\s+d[ií]as?\b", re.IGNORECASE)),
    ("prorroga", re.compile(r"\bpr[oó]rroga\b", re.IGNORECASE)),
]
res_patterns = [
    ("lcsp", re.compile(r"\bLCSP\b|\bLey\s+9/2017\b|\bart[íi]culo(?:s)?\b", re.IGNORECASE)),
    ("causas", re.compile(r"\bcausas?\s+de\s+resoluci[oó]n\b", re.IGNORECASE)),
    ("incumplimiento", re.compile(r"\bincumplim", re.IGNORECASE)),
    ("rescision", re.compile(r"\brescisi[oó]n\b|\brescind", re.IGNORECASE)),
    ("extincion", re.compile(r"\bextinci[oó]n\b", re.IGNORECASE)),
    ("mutuo_acuerdo", re.compile(r"\bmutuo\s+acuerdo\b", re.IGNORECASE)),
]

def pick_varied(label: str, k: int, patterns):
    xs = pool[label].copy()
    random.shuffle(xs)
    picked = []
    used_uids = set()

    # 1) uno por patrón si se puede
    for name, rx in patterns:
        if len(picked) >= k:
            break
        for ex in xs:
            if ex["doc_uid"] in used_uids:
                continue
            if rx.search(ex["text"]):
                picked.append(ex)
                used_uids.add(ex["doc_uid"])
                break

    # 2) completar
    for ex in xs:
        if len(picked) >= k:
            break
        if ex["doc_uid"] in used_uids:
            continue
        picked.append(ex)
        used_uids.add(ex["doc_uid"])

    return picked[:k]

for main, label, patterns in [
    ("P4", P_TO_LABEL["P4"], dur_patterns),
    ("P9", P_TO_LABEL["P9"], res_patterns),
]:
    for ex in pick_varied(label, k=4, patterns=patterns):
        add_example(label, FIXED_CRITERION[main], ex)

print("memory_selected tamaño:", len(memory_selected))
print("Docs únicos usados:", len(used_doc_uids))


memory_selected tamaño: 16
Docs únicos usados: 16


In [18]:
#validacion offsets y guardado de memoria 
errors = 0
for ex in memory_selected:
    uid = str(ex["doc_uid"])
    txt = key_to_text.get(key, "")
    s, e = ex["start"], ex["end"]
    if not txt or not (0 <= s < e <= len(txt)) or not txt[s:e].strip():
        errors += 1

print("Errores de validación:", errors)


# Preview (robusto: doc_uid puede ser int o str)
for i, ex in enumerate(memory_selected[:16], 1):
    uid = str(ex["doc_uid"])              # <- clave como string
    txt = key_to_text.get(key, "")        # <- evita KeyError
    span = txt[ex["start"]:ex["end"]].replace("\n", " ") if txt else ""

    print(f"\n[{i}] {ex['label']} :: {ex['criterion']}")
    print("doc_uid:", uid[:12] + "..." if len(uid) > 12 else uid)
    print("span:", span[:260])

PATH_MEM_CURATED = OUT_DIR / "memory_selected_CURATED.json"
PATH_MEM_CURATED.write_text(json.dumps(memory_selected, ensure_ascii=False, indent=2), encoding="utf-8")
print("\nGuardado:", PATH_MEM_CURATED)


Errores de validación: 16

[1] OBJETO :: P1.MAIN (ejemplo a nivel etiqueta) — Se anota como “objeto” la información resumida y concisa que describe la prestación principal del contrato definida en el título, antecedentes o en las cláusulas. Se anota el objeto tantas veces como aparezca en el contrato. Se anota como “objeto” los lotes identificados que acompañan el objeto del contrato. (gold)
doc_uid: -474927492
span: 

[2] OBJETO :: P1.1 Inclusión de expedientes en objeto — Se anotan como “objeto” los números de expediente que acompañan al objeto del contrato. Dicho expediente debe encontrarse adyacente (precediendo o siguiendo) a la definición de la prestación principal. (gold)
doc_uid: 607056704
span: 

[3] OBJETO :: P1.2 Inclusión del lote en el objeto — Se anota como “objeto” los lotes identificados que acompañan el objeto del contrato (gold)
doc_uid: 954871603
span: 

[4] OBJETO :: P1.3 Delimitación del objeto si está entrecomillado — Si el objeto está entrecomillado o con comilla

In [19]:
#Selección de memoria (1 ejemplo por criterio/subtipo)
#Aquí definimos exactamente lo que irá al prompt en Experimento 2/3 - enfoque híbrido, manual/semi-automático

# Rellenamos esta lista a mano con los ejemplos que elijamos 
# (doc_uid + start/end + label + criterion)
memory_selected = [
    # Ejemplo (rellenamos con los elegidos):
    # {
    #   "label": "PRECIO_DEL_CONTRATO",
    #   "criterion": "P1.2",
    #   "doc_uid": "<sha1>",
    #   "start": 2877,
    #   "end": 3026
    # },
]


## **README_guidelines_MVP_defs**

## Estructura de criterios y subtipos en la guía MVP

Las etiquetas incluidas en esta guía MVP no presentan una estructura homogénea de subtipos.

### Etiquetas con subtipos conceptuales explícitos
Las etiquetas **P1. Objeto** y **P2. Precio del contrato** incluyen:

- Un **criterio general** aplicable a la etiqueta principal.
- **Subtipos explícitos** (por ejemplo, P1.1, P1.2, P2.1, etc.) que:
  - introducen **criterios adicionales o específicos**,
  - delimitan casos particulares de anotación,
  - y requieren ejemplos diferenciados.

En estos casos, la memoria y los ejemplos se organizan **por subtipo**, ya que cada subtipo representa una variación funcional relevante del segmento jurídico anotado.

### Etiquetas sin subtipos conceptuales diferenciados
Las etiquetas **P4. Duración del contrato** y **P9. Resolución** presentan:

- Un **único criterio general** por etiqueta.
- Subdivisiones internas (por ejemplo, P4.01, P4.02, P9.03, etc.) que:
  - **no introducen nuevos criterios de anotación**,
  - sino que agrupan **variantes ejemplificadas** de la misma función jurídica (fechas, plazos, fórmulas, referencias legales, etc.).

En estos casos, los subtipos se consideran **variantes de realización textual**, no categorías conceptuales independientes. Por tanto, los ejemplos se emplean para ilustrar diversidad expresiva, sin modificar el criterio de anotación.

## Uso de los ejemplos de la guía y prevención de leakage

Todos los ejemplos incluidos en esta guía MVP aparecen entre corchetes `[...]` y corresponden a citas literales de contratos reales.

Para evitar leakage entre entrenamiento, memoria y evaluación:

- Los documentos que contienen cualquiera de estos extractos literales se excluyen de:
  - `prompt_regression`
  - `validation`
  - `test`
- Dichos documentos pueden mantenerse en el conjunto de entrenamiento, pero **nunca se emplean para evaluación**, dado que los ejemplos de la guía se utilizan como memoria o few-shot en el prompt del agente.

Esta separación garantiza que el agente no sea evaluado sobre segmentos contractuales que ha visto explícitamente como ejemplos.

**Cómo reflejar esto en el código / estructuras (para que no se  olvide)**

Te recomiendo que, en la memoria (memory_examples.json), sigas esta convención:

- Para P1 / P2:

{
  "label": "OBJETO",
  "criterion": "P1.2 Inclusión del lote en el objeto",
  ...
}


- Para P4 / P9:

{
  "label": "DURACION_TOTAL_DEL_CONTRATO",
  "criterion": "P4 (criterio general)",
  ...
}


O, si quieres ser aún más explícita:

{
  "label": "DURACION_TOTAL_DEL_CONTRATO",
  "criterion": "P4 – criterio único (subtipos solo ejemplificativos)",
  ...
}


**Eso hace que:**
- el LLM no “busque” diferencias conceptuales donde no las hay,
- y tú puedas justificar por qué no hay un ejemplo “por subtipo” en P4/P9.

**Justificación científica (por si la necesitas en paper o memoria)**

“Los subtipos definidos en la guía se interpretan como categorías conceptuales únicamente cuando introducen criterios de anotación diferenciados; en caso contrario, se tratan como variantes de realización textual de una misma función jurídica.”

**Resumen claro**

✔️ No todas las etiquetas necesitan subtipos con criterio
✔️ P1 y P2 sí → subtipos conceptuales
✔️ P4 y P9 no → subtipos ejemplificativos
✔️ Lo documentas explícitamente → no hay ambigüedad
✔️ El agente aprende qué fijarse (criterio) y qué es solo variación formal

Continuamos por donde íbamos:
**En NB03, el siguiente tramo lógico (ya con guidelines_MVP_defs pulida) es:**

- Extraer, desde guidelines_MVP_defs, los ejemplos entre corchetes [...] y guardarlos como “citas prohibidas” (para evitar leakage).
- Localizar esas citas en el gold (Corpus_anotado) y construir un fichero removed_doc_uids.json con los docs que las contienen.
- Rehacer el “Explorador de candidatos desde el gold”, pero excluyendo esos removed_doc_uids para que lo que elijas como memoria no contamine val/test/prompt_regression.

Esto encaja con tu decisión híbrida: memoria/few-shot desde gold (offsets), y exclusión de evaluación solo para documentos que contienen exactamente los extractos usados en la memoria.

A continuación te dejo las celdas “limpias” para pegar en NB03, retomando justo donde dices. (Las asumo después de que ya tienes gold_mvp listo y con doc_uid, doc_id, text, y spans filtrados a tus 4 etiquetas; el formato de tags como lista de dict con start/end/tag es el que tú ya confirmaste, y el corpus gold está en Corpus_anotado.json

**Con esto ya tenemos la garantía de que lo que estamos eligiendo para memoria no pertenece a docs contaminados por citas de las guías.**
Siguientes pasos:

1. Lee config/guidelines_MVP_defs.txt
2. Extrae tipos (P1, P2, P4, P9) y subtipos (P1.1, P1.2, P4.01, etc.)
3. Extrae el Criterio: cuando existe (P1/P2 y sus subtipos)
4. Carga tu gold (gold/corpus_annotated.json o .jsonl, ajustable)
5. Construye memory_selected eligiendo un ejemplo aleatorio por tipo y subtipo (con offsets correctos)
6. Asigna criterion así:
    - P1/P2: usa el texto “Criterio:” real del txt (si lo encuentra)
    - P4/P9: usa el criterio explícito que acordamos:
        - "P4 – criterio único (subtipos solo ejemplificativos)"
        - "P9 – criterio único (subtipos solo ejemplificativos)"

**Nota importante (metodológica): como la selección es aleatoria, no garantiza que el span elegido “represente” semánticamente el subtipo. Sirve para validar pipeline y formato. Luego sustituyes por ejemplos verdaderamente alineados.**

Tendríamos 2 formas de hacer la selección: manual de memoria (con mi criterion explícito para P4/P9) o automáticamente (random). Optamos por random



In [20]:
#debug para ver qué ve el parser 
from pathlib import Path
import re

ROOT = Path("/home/jovyan/inesagent")
PATH_GUIDE_TXT = ROOT / "config" / "guidelines_MVP_defs.txt"

raw = PATH_GUIDE_TXT.read_text(encoding="utf-8", errors="ignore")
raw = raw.lstrip("\ufeff").replace("\r\n", "\n").replace("\r", "\n").replace("\xa0", " ")

print("LEN:", len(raw))
print("Primeros 200 repr:", repr(raw[:800]))

# ¿Hay cabeceras y subtipos en el texto?
print("Mains encontrados (P1./P2./P4./P9.):", len(re.findall(r"(?m)^\s*P[1249]\.\s+\S+", raw)))
print("Subtipos encontrados tipo P1.1:", len(re.findall(r"(?m)^\s*P[1249]\.\d{1,2}\b", raw)))
print("Subtipos encontrados tipo P4.01:", len(re.findall(r"(?m)^\s*P[1249]\.\d{2}\b", raw)))

# Muestra 20 líneas candidatas que empiezan por P algo
cand = re.findall(r"(?m)^\s*(P[1249]\.[^\n]{0,80})$", raw)
print("\nEjemplos de líneas P*:")
for x in cand[:50]:
    print("-", repr(x))


LEN: 14078
Primeros 200 repr: 'Guía de anotación MVP_defs (P1, P2, P4, P9)\nP1. Objeto\nCriterio: Se anota como “objeto” la información resumida y concisa que describe la prestación principal del contrato definida en el título, antecedentes o en las cláusulas. Se anota el objeto tantas veces como aparezca en el contrato. Se anota como “objeto” los lotes identificados que acompañan el objeto del contrato. \nEjemplos: El objeto del presente contrato es la realización, por parte del contratista, del [suministro de armarios consigna con apertura individual mediante cerradura electrónica para los edificios del Banco de España en Lumbreras] de acuerdo con las especificaciones contenidas en el Pliego de Prescripciones Técnicas. \nP1.1 Inclusión de expedientes en objeto \nCriterio: Se anotan como “objeto” los números de expediente q'
Mains encontrados (P1./P2./P4./P9.): 4
Subtipos encontrados tipo P1.1: 27
Subtipos encontrados tipo P4.01: 21

Ejemplos de líneas P*:
- 'P1. Objeto'
- 'P1.1 Inclu

In [21]:

# NB03 — Baseline: construir memory_selected_AUTO.json (random/heurístico) desde guidelines_MVP_defs.txt + gold
# NB03 — Baseline: construir memory_selected_AUTO.json (random/heurístico) desde guidelines_MVP_defs.txt + gold

from pathlib import Path
import json, re, hashlib, random
from collections import defaultdict

# =========================
# 0) Rutas + seed
# =========================
ROOT = Path("/home/jovyan/inesagent")
PATH_GUIDE_TXT = ROOT / "config" / "guidelines_MVP_defs.txt"

PATH_GOLD_JSON  = ROOT / "gold" / "corpus_annotated.json"    # opcional
PATH_GOLD_JSONL = ROOT / "gold" / "corpus_annotated.jsonl"   # recomendado

OUT_DIR = ROOT / "outputs" / "memory"
OUT_DIR.mkdir(parents=True, exist_ok=True)

SEED = 42
random.seed(SEED)

# =========================
# 1) Utilidades
# =========================
def stable_uid(text: str) -> str:
    return hashlib.sha1(text.encode("utf-8")).hexdigest()

def load_json(path: Path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def load_jsonl(path: Path):
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                rows.append(json.loads(line))
    return rows

def norm_all(text: str) -> str:
    # normaliza BOM, CRLF/CR y NBSP
    return text.lstrip("\ufeff").replace("\r\n", "\n").replace("\r", "\n").replace("\xa0", " ")

def norm_line(s: str) -> str:
    return " ".join((s or "").split())

# MVP labels
P_TO_LABEL = {
    "P1": "OBJETO",
    "P2": "PRECIO_DEL_CONTRATO",
    "P4": "DURACION_TOTAL_DEL_CONTRATO",
    "P9": "RESOLUCION",
}
MVP_LABELS = set(P_TO_LABEL.values())

FIXED_CRITERION = {
    "P4": "P4 – criterio único (subtipos solo ejemplificativos)",
    "P9": "P9 – criterio único (subtipos solo ejemplificativos)",
}

# =========================
# 2) Parsear guía TXT (mains + subtipos + criterio)
# =========================
if not PATH_GUIDE_TXT.exists():
    raise FileNotFoundError(f"No encuentro {PATH_GUIDE_TXT}")

raw_txt = norm_all(PATH_GUIDE_TXT.read_text(encoding="utf-8", errors="ignore"))
lines = [ln.strip() for ln in raw_txt.split("\n") if ln.strip()]

RE_MAIN = re.compile(r"^\s*(P(?:1|2|4|9))\.\s+(.+?)\s*$", re.IGNORECASE)
RE_SUB  = re.compile(r"^\s*(P(?:1|2|4|9))\s*\.\s*(\d{1,2})\s*(.*)\s*$", re.IGNORECASE)
RE_CRIT = re.compile(r"^\s*Criterio\s*:\s*(.+?)\s*$", re.IGNORECASE)

def fmt_subcode(main: str, num: str) -> str:
    n = int(num)
    # P4/P9 suelen ir como 2 dígitos (P4.01, P9.12), P1/P2 como 1 dígito (P1.1)
    if main in {"P4","P9"}:
        return f"{main}.{n:02d}"
    return f"{main}.{n}"

guide = {k: {"title": "", "criterion": "", "subtypes": {}} for k in ["P1","P2","P4","P9"]}
current_code = None

def ensure_sub(main, subcode):
    guide[main]["subtypes"].setdefault(subcode, {"title": "", "criterion": ""})

for ln in lines:
    ln = norm_line(ln)

    m1 = RE_MAIN.match(ln)
    if m1:
        main = m1.group(1).upper()
        guide[main]["title"] = m1.group(2).strip()
        current_code = main
        continue

    m2 = RE_SUB.match(ln)
    if m2:
        main = m2.group(1).upper()
        num  = m2.group(2)
        title = (m2.group(3) or "").strip()  # puede estar vacío (P4.01 / P9.03...)
        subcode = fmt_subcode(main, num)
        ensure_sub(main, subcode)
        guide[main]["subtypes"][subcode]["title"] = title
        current_code = subcode
        continue

    m3 = RE_CRIT.match(ln)
    if m3 and current_code:
        crit = m3.group(1).strip()
        if "." in current_code:
            main = current_code.split(".")[0]
            ensure_sub(main, current_code)
            guide[main]["subtypes"][current_code]["criterion"] = crit
        else:
            guide[current_code]["criterion"] = crit
        continue

print("Tipos detectados:", [k for k in guide.keys() if guide[k]["title"]])
for k in ["P1","P2","P4","P9"]:
    subs = sorted(guide[k]["subtypes"].keys())
    print(f"{k} subtipos:", len(subs), "| ejemplo:", subs[:6])

# =========================
# 3) Cargar GOLD (COMPLETO)
# =========================
if PATH_GOLD_JSONL.exists():
    gold = load_jsonl(PATH_GOLD_JSONL)
elif PATH_GOLD_JSON.exists():
    gold = load_json(PATH_GOLD_JSON)
else:
    raise FileNotFoundError("No encuentro gold/corpus_annotated.jsonl ni .json")

print("Docs gold:", len(gold))

# =========================
# 4) Pool de spans por etiqueta
# =========================
pool = {lab: [] for lab in MVP_LABELS}

key_to_text = {}
for d in gold:
    text = d.get("text", "")
    if not text:
        continue
    uid = stable_uid(text)
    key_to_text[uid] = text
    doc_id = d.get("id")

    for t in d.get("tags", []):
        lab = t.get("tag")
        if lab not in MVP_LABELS:
            continue
        s = int(t["start"]); e = int(t["end"])
        if not (0 <= s < e <= len(text)):
            continue
        span_txt = text[s:e]
        if not span_txt.strip():
            continue
        pool[lab].append({"doc_id": doc_id, "doc_uid": uid, "start": s, "end": e, "text": span_txt})

print("Spans en pool por etiqueta:")
for lab in sorted(pool.keys()):
    print(" -", lab, ":", len(pool[lab]))

def pick_random_span(label: str):
    if not pool[label]:
        raise ValueError(f"No hay spans en pool para {label}")
    return random.choice(pool[label])

# =========================
# 5) Construir memory_selected AUTO
#    - P1/P2: 1 por main + 1 por subtipo (random)
#    - P4/P9: 4 por variedad formal (heurística simple)
# =========================
memory_selected = []

# 5.1 MAIN (P1/P2/P4/P9)
for main in ["P1","P2","P4","P9"]:
    label = P_TO_LABEL[main]
    ex = pick_random_span(label)
    if main in FIXED_CRITERION:
        criterion = FIXED_CRITERION[main]
    else:
        criterion = f"{main} {guide[main]['title']} — {guide[main]['criterion']}".strip(" —")
    memory_selected.append({"label": label, "criterion": criterion, "doc_uid": ex["doc_uid"], "start": ex["start"], "end": ex["end"]})

# 5.2 SUBTIPOS (solo P1/P2; P4/P9 los tratamos como variedad formal)
for main in ["P1","P2"]:
    label = P_TO_LABEL[main]
    for subcode, subinfo in sorted(guide[main]["subtypes"].items()):
        ex = pick_random_span(label)
        title = (subinfo.get("title") or "").strip()
        crit  = (subinfo.get("criterion") or "").strip()
        criterion = f"{subcode} {title} — {crit}".strip(" —")
        memory_selected.append({"label": label, "criterion": criterion, "doc_uid": ex["doc_uid"], "start": ex["start"], "end": ex["end"]})

# 5.3 Variedad formal P4/P9: 4 ejemplos cada una (sin subtipos)
def pick_variety(label: str, patterns: list, k: int = 4):
    candidates = pool[label].copy()
    random.shuffle(candidates)
    picked, used_uids = [], set()

    for _, rx in patterns:
        if len(picked) >= k:
            break
        for ex in candidates:
            if ex["doc_uid"] in used_uids:
                continue
            if rx.search(ex["text"]):
                picked.append(ex)
                used_uids.add(ex["doc_uid"])
                break

    # completar si falta (sin repetir doc_uid)
    for ex in candidates:
        if len(picked) >= k:
            break
        if ex["doc_uid"] in used_uids:
            continue
        picked.append(ex)
        used_uids.add(ex["doc_uid"])

    return picked

dur_patterns = [
    ("formalizacion", re.compile(r"\bformalizaci[oó]n\b", re.IGNORECASE)),
    ("fechas", re.compile(r"\b\d{1,2}\s+de\s+[a-záéíóú]+\s+de\s+\d{4}\b", re.IGNORECASE)),
    ("anos", re.compile(r"\b\d+\s+a[nñ]os?\b", re.IGNORECASE)),
    ("meses", re.compile(r"\b\d+\s+mes(?:es)?\b", re.IGNORECASE)),
    ("dias", re.compile(r"\b\d+\s+d[ií]as?\b", re.IGNORECASE)),
    ("prorroga", re.compile(r"\bpr[oó]rroga\b", re.IGNORECASE)),
]
res_patterns = [
    ("lcsp", re.compile(r"\bLCSP\b|\bLey\s+9/2017\b|\bart[íi]culo(?:s)?\b", re.IGNORECASE)),
    ("causas", re.compile(r"\bcausas?\s+de\s+resoluci[oó]n\b", re.IGNORECASE)),
    ("incumplimiento", re.compile(r"\bincumplim", re.IGNORECASE)),
    ("rescision", re.compile(r"\brescisi[oó]n\b|\brescind", re.IGNORECASE)),
    ("extincion", re.compile(r"\bextinci[oó]n\b", re.IGNORECASE)),
    ("mutuo_acuerdo", re.compile(r"\bmutuo\s+acuerdo\b", re.IGNORECASE)),
]

for ex in pick_variety(P_TO_LABEL["P4"], dur_patterns, k=4):
    memory_selected.append({"label": P_TO_LABEL["P4"], "criterion": FIXED_CRITERION["P4"], "doc_uid": ex["doc_uid"], "start": ex["start"], "end": ex["end"]})
for ex in pick_variety(P_TO_LABEL["P9"], res_patterns, k=4):
    memory_selected.append({"label": P_TO_LABEL["P9"], "criterion": FIXED_CRITERION["P9"], "doc_uid": ex["doc_uid"], "start": ex["start"], "end": ex["end"]})

# =========================
# 6) Validación rápida
# =========================
errors = 0
for ex in memory_selected:
    uid = ex["doc_uid"]
    txt = key_to_text.get(uid)
    if not txt:
        errors += 1
        continue
    s, e = ex["start"], ex["end"]
    if not (0 <= s < e <= len(txt)):
        errors += 1
        continue
    if not txt[s:e].strip():
        errors += 1

print("\nTotal memory_selected_AUTO:", len(memory_selected))
print("Errores de validación:", errors)

OUT_AUTO = OUT_DIR / "memory_selected_AUTO.json"
OUT_AUTO.write_text(json.dumps(memory_selected, ensure_ascii=False, indent=2), encoding="utf-8")
print("Guardado:", OUT_AUTO)


Tipos detectados: ['P1', 'P2', 'P4', 'P9']
P1 subtipos: 3 | ejemplo: ['P1.1', 'P1.2', 'P1.3']
P2 subtipos: 3 | ejemplo: ['P2.1', 'P2.2', 'P2.3']
P4 subtipos: 9 | ejemplo: ['P4.01', 'P4.02', 'P4.03', 'P4.04', 'P4.05', 'P4.06']
P9 subtipos: 12 | ejemplo: ['P9.01', 'P9.02', 'P9.03', 'P9.04', 'P9.05', 'P9.06']
Docs gold: 373
Spans en pool por etiqueta:
 - DURACION_TOTAL_DEL_CONTRATO : 468
 - OBJETO : 925
 - PRECIO_DEL_CONTRATO : 436
 - RESOLUCION : 526

Total memory_selected_AUTO: 18
Errores de validación: 0
Guardado: /home/jovyan/inesagent/outputs/memory/memory_selected_AUTO.json


Perfecto, lo que vemos al ejecutarla:
- Cuántos tipos (P1/P2/P4/P9) detecta en la guía
- Cuántos subtipos detecta por cada tipo
- Cuántos spans hay por cada etiqueta en el gold

Un memory_selected_AUTO.json ya creado con:
- label
- criterion (con tu convención explícita en P4/P9)
- doc_uid
- start/end

**SIGUIENTE AJUSTE (para que ya sea “anti-leakage” de verdad)**
- Cambiar selección random por selección semánticamente alineada (tú eliges).
- Construir blocked_doc_uids_by_memory.json con los doc_uid usados en la memoria (y solo esos), para excluirlos de val/test/prompt_regression.

Hemos detectado:
- P1: 3 subtipos
- P2: 3 subtipos
- P4: 6 subtipos (ejemplificativos)
- P9: 12 subtipos (ejemplificativos)

Si seleccionamos **_1 ejemplo por subtipo_**, te vas a meter 3+3+6+12 = **24 ejemplos en memoria, que para un 7B puede ser demasiado**.

Para MVP, haz esto:
- P1: 1 ejemplo para P1 (general) + 1 por cada subtipo (3) → 4 ejemplos
- P2: 1 ejemplo para P2 (general) + 1 por cada subtipo (3) → 4 ejemplos
- P4: criterio único → selecciona 3–4 ejemplos que cubran variedad (fecha fija, duración en meses/años, “desde formalización”, “plazo máximo”, prórroga)
- P9: criterio único → selecciona 3–4 ejemplos (causas LCSP, resolución por incumplimiento, rescisión automática, “extinción”, etc.)

**Total memoria recomendada MVP: 14–16 ejemplos.**

**Qué significa el recuento**
Esto NO significa documentos; significa spans anotados en el gold. Dado que antes vimos:
- 373 documentos con al menos un span MVP

la lectura correcta es:

Cada documento suele tener:
- 1–3 OBJETO (título + cláusula + lote, etc.)
- 1–2 RESOLUCION
- 1–2 DURACION
- 1 PRECIO (a veces más si hay anualidades)

➡️ Es exactamente el patrón jurídico esperado.
➡️ No hay desbalance patológico.
➡️ No necesitas sampling agresivo.

**¿Es demasiado para un MVP?**
No. Al contrario:
- Para prompting: usarás solo 14–16 ejemplos en memoria.
- Para evaluación: tendrás suficientes casos variados para medir overlap y exact match con estabilidad estadística.
- Para generalización: la diversidad interna es alta (bien).

373 docs es una base sólida para:
- un MVP serio,
- un paper corto,
- o una demo reproducible.

**Decisión clave (ya podemos fijarla)**

Con estos números, te recomiendo cerrar ya estas decisiones:

**✔️ Memoria / few-shot (prompt)**

**OBJETO (P1)**
- 1 ejemplo general
- 1 por cada subtipo (3)
- → 4 ejemplos

**PRECIO (P2)**
- 1 ejemplo general
- 1 por cada subtipo (3)
- → 4 ejemplos

**DURACIÓN (P4)**
criterio único
- 3–4 ejemplos representativos

**RESOLUCIÓN (P9)**
criterio único
- 3–4 ejemplos representativos


🔢 Total memoria: 14–16 ejemplos
Perfecto para Mistral 7B / Llama 3.1 8B.

**Sustituimos la memoria aleatoria por una memoria curada**
Genera memory_selected curada automáticamente siguiendo tu metodología:
- P1 / P2: alineación por criterio/subtipo usando la guía (guidelines_MVP_defs.docx):
    - Para cada subtipo (Heading 3) busca un ejemplo entre corchetes [...]
    - Localiza ese extracto literal en el gold
    - Recupera el span anotado (offsets) que lo cubre y lo añade con criterion = "<código subtipo> <título> — <criterio>"

- P4 / P9: selección por variedad formal (no por subtipo), con buckets heurísticos (fechas, “formalización”, meses/años/días, prórroga, LCSP, causas, etc.) y criterion fijo:
    - "P4 – criterio único (subtipos solo ejemplificativos)"
    - "P9 – criterio único (subtipos solo ejemplificativos)"

Al final te deja:
- memory_selected listo para copiar/pegar (es una lista de dicts con label/criterion/doc_uid/start/end)
- y guarda outputs/memory/memory_selected_CURATED.json

Nota: si algún subtipo no se puede resolver (porque el extracto literal no aparece tal cual en el gold, o no cae dentro de un span anotado), el código hace fallback a un span aleatorio de esa etiqueta y lo marca explícitamente en criterion con “(fallback)”.


**!! DISCLAIMER**: con la siguiente celda, capturamos todos los quotes de las guías en el gold. Resulta que **algunos de los ejemplos que aparecen en las guías no está en el gold** (se ve que no los incluyeron en el corpus). ejemplos: 

`ValueError: [SIN FALLBACK] No encuentro el quote de P1.MAIN en GOLD.
Quote: **suministro de armarios consigna con apertura individual mediante cerradura electrónica para los edificios del Banco de España en Lumbreras**`

`ValueError: [SIN FALLBACK] No encuentro el quote del subtipo P2.1 en GOLD.
Quote: 29999.53 € (IVA incuido)`

Así que como queremos la memoria curada sin fallbacks, lo correcto es **si no se puede alinear, NO lo incluimos** (y seguimos con el resto P1.1/P1.2/P1.3). Esto sigue cumpliendo la metodología, porque el “main” es redundante: los subtipos ya cubren el concepto, y además P4/P9 ya van por criterio único.

Solución mínima (y metodológicamente limpia): si estamos en modo "sin fallback" y el gold es un subset (373 docs), tenemos que tratar como "opcioales" TODOS los subtipos cuyo ejemplo literal no esté presente en este subset.
- P1/P2: intentar alinear por subtipo con cita literal;
    - si no se encuentra en el gold actual → omitimos ese subtipo con warning (igual que hicimos con .MAIN), sin fallback.
- P4/P9: seguir como estabas (variedad formal por heurísticas), sin depender de la guía para encontrar citas.

Añadimos parche  para sustituir el bucle de P1/P2 “por subtipos” (incluye MAIN y subtipos) sin cambiar el resto.

In [22]:
#construimos memory_selected curada
# NB03 — Construcción de memoria curada (SIN fallbacks) desde guidelines_MVP_defs.txt + gold
# NB03 — Construcción de memoria curada (SIN fallbacks) desde guidelines_MVP_defs.txt + gold

from pathlib import Path
import json, re, hashlib, random
from collections import defaultdict

ROOT = Path("/home/jovyan/inesagent")
PATH_GUIDE_TXT = ROOT / "config" / "guidelines_MVP_defs.txt"

PATH_GOLD_JSON  = ROOT / "gold" / "corpus_annotated.json"
PATH_GOLD_JSONL = ROOT / "gold" / "corpus_annotated.jsonl"

OUT_DIR = ROOT / "outputs" / "memory"
OUT_DIR.mkdir(parents=True, exist_ok=True)

SEED = 42
random.seed(SEED)

P_TO_LABEL = {
    "P1": "OBJETO",
    "P2": "PRECIO_DEL_CONTRATO",
    "P4": "DURACION_TOTAL_DEL_CONTRATO",
    "P9": "RESOLUCION",
}
MVP_LABELS = set(P_TO_LABEL.values())

FIXED_CRITERION = {
    "P4": "P4 – criterio único (subtipos solo ejemplificativos)",
    "P9": "P9 – criterio único (subtipos solo ejemplificativos)",
}

BRACKET_RE = re.compile(r"\[(.+?)\]")

def stable_uid(text: str) -> str:
    return hashlib.sha1(text.encode("utf-8")).hexdigest()

def doc_key(d: dict, txt: str) -> str:
    """Clave estable del documento.
    - Si el gold trae id (int/str), usamos ese id.
    - Si no, usamos un hash sha1 del texto.
    Siempre devolvemos str.
    """
    if "id" in d and d["id"] is not None and str(d["id"]).strip() != "":
        return str(d["id"])
    return stable_uid(txt)

def load_json(path: Path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

def load_jsonl(path: Path):
    rows = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                rows.append(json.loads(line))
    return rows

def norm_all(text: str) -> str:
    return text.lstrip("\ufeff").replace("\r\n", "\n").replace("\r", "\n").replace("\xa0", " ")

def normalize_for_match(s: str) -> str:
    # normalización robusta para matching
    s = (s or "").lower()
    s = " ".join(s.split())
    return s

# =========================
# 1) Parsear guía TXT: mains/subtipos/criterio + ejemplos [...]
# =========================
if not PATH_GUIDE_TXT.exists():
    raise FileNotFoundError(f"No encuentro {PATH_GUIDE_TXT}")

raw_txt = norm_all(PATH_GUIDE_TXT.read_text(encoding="utf-8", errors="ignore"))
lines = [ln.strip() for ln in raw_txt.split("\n") if ln.strip()]

RE_MAIN = re.compile(r"^\s*(P(?:1|2|4|9))\.\s+(.+?)\s*$", re.IGNORECASE)
RE_SUB  = re.compile(r"^\s*(P(?:1|2|4|9))\s*\.\s*(\d{1,2})\s*(.*)\s*$", re.IGNORECASE)
RE_CRIT = re.compile(r"^\s*Criterio\s*:\s*(.+?)\s*$", re.IGNORECASE)

def fmt_subcode(main: str, num: str) -> str:
    n = int(num)
    if main in {"P4","P9"}:
        return f"{main}.{n:02d}"
    return f"{main}.{n}"

guide = {k: {"title": "", "criterion": "", "subtypes": {}} for k in ["P1","P2","P4","P9"]}
current_code = None

def ensure_sub(main, subcode):
    guide[main]["subtypes"].setdefault(subcode, {"title": "", "criterion": "", "examples": []})

for ln in lines:
    ln_norm = " ".join(ln.split())

    m1 = RE_MAIN.match(ln_norm)
    if m1:
        main = m1.group(1).upper()
        guide[main]["title"] = m1.group(2).strip()
        current_code = main
        continue

    m2 = RE_SUB.match(ln_norm)
    if m2:
        main = m2.group(1).upper()
        num  = m2.group(2)
        title = (m2.group(3) or "").strip()
        subcode = fmt_subcode(main, num)
        ensure_sub(main, subcode)
        guide[main]["subtypes"][subcode]["title"] = title
        current_code = subcode
        continue

    m3 = RE_CRIT.match(ln_norm)
    if m3 and current_code:
        crit = m3.group(1).strip()
        if "." in current_code:
            main = current_code.split(".")[0]
            ensure_sub(main, current_code)
            guide[main]["subtypes"][current_code]["criterion"] = crit
        else:
            guide[current_code]["criterion"] = crit
        continue

    # ejemplos entre corchetes en cualquier línea -> se asignan al bloque actual (main o subtipo)
    for m in BRACKET_RE.finditer(ln):
        ex = m.group(1).strip()
        if not ex:
            continue
        if current_code and "." in current_code:
            main = current_code.split(".")[0]
            ensure_sub(main, current_code)
            guide[main]["subtypes"][current_code]["examples"].append(ex)
        elif current_code in guide:
            pseudo = f"{current_code}.MAIN"
            ensure_sub(current_code, pseudo)
            guide[current_code]["subtypes"][pseudo]["title"] = "(ejemplo a nivel etiqueta)"
            guide[current_code]["subtypes"][pseudo]["criterion"] = guide[current_code]["criterion"]
            guide[current_code]["subtypes"][pseudo]["examples"].append(ex)

print("Tipos detectados:", [k for k in guide.keys() if guide[k]["title"]])
for k in ["P1","P2","P4","P9"]:
    subs = sorted(guide[k]["subtypes"].keys())
    main_pseudo = 1 if f"{k}.MAIN" in guide[k]["subtypes"] else 0
    real_subs = [s for s in subs if not s.endswith(".MAIN")]
    print(f"{k} subtipos:", len(real_subs), "| +MAIN_pseudo:", main_pseudo, "| ejemplo:", real_subs[:15])

# =========================
# 2) Cargar GOLD COMPLETO + construir índices + pool MVP
# =========================
if PATH_GOLD_JSONL.exists():
    gold = load_jsonl(PATH_GOLD_JSONL)
elif PATH_GOLD_JSON.exists():
    gold = load_json(PATH_GOLD_JSON)
else:
    raise FileNotFoundError("No encuentro gold/corpus_annotated.jsonl ni .json")

print("Docs gold:", len(gold))

key_to_text = {}
key_to_tags = {}
for d in gold:
    txt = d.get("text", "")
    if not txt:
        continue
    key = doc_key(d, txt)
    key_to_text[key] = txt
    key_to_tags[key] = d.get("tags", [])

# Pool de spans MVP
pool = defaultdict(list)
for key, txt in key_to_text.items():
    for t in key_to_tags.get(key, []):
        lab = t.get("tag")
        if lab not in MVP_LABELS:
            continue
        s = int(t["start"]); e = int(t["end"])
        if 0 <= s < e <= len(txt):
            span_txt = txt[s:e]
            if span_txt.strip():
                pool[lab].append({"id": key, "start": s, "end": e, "text": span_txt})

print("Spans por etiqueta en gold (pool):")
for lab in sorted(pool.keys()):
    print(" -", lab, ":", len(pool[lab]))

# =========================
# 3) Matching robusto quote -> (uid, qstart, qend) + recuperar span anotado
# =========================
gold_norm = [(key, normalize_for_match(txt)) for key, txt in key_to_text.items()]

def find_quote_in_gold_robust(quote: str):
    qn = normalize_for_match(quote)
    if not qn:
        return None

    # (A) búsqueda por contención en texto normalizado
    for key, tnorm in gold_norm:
        if qn in tnorm:
            txt = key_to_text[key]

            # (B) intentamos match exacto en original
            pos = txt.find(quote)
            if pos != -1:
                return key, pos, pos + len(quote)

            # (C) regex flexible por tokens (espacios variables)
            toks = [re.escape(tok) for tok in quote.split() if tok.strip()]
            if len(toks) >= 4:
                pat = r"\s+".join(toks)
                m = re.search(pat, txt, flags=re.IGNORECASE)
                if m:
                    return key, m.start(), m.end()

            # Si vimos contención normalizada pero no podemos mapear offsets, seguimos buscando
    return None

def pick_annotated_span_covering(key: str, label: str, q_start: int, q_end: int):
    txt = key_to_text[key]
    best = None
    best_score = -1

    for t in key_to_tags.get(key, []):
        if t.get("tag") != label:
            continue
        s = int(t["start"]); e = int(t["end"])
        if not (0 <= s < e <= len(txt)):
            continue

        overlap = max(0, min(e, q_end) - max(s, q_start))
        covers = (s <= q_start and q_end <= e)
        score = (10_000 + overlap) if covers else overlap

        if overlap > 0 and score > best_score:
            best_score = score
            best = (s, e)

    return best

# =========================
# 4) Construir memoria curada SIN fallback
# =========================
memory_selected = []
used_keys = set()

def add_example(label, criterion, key, s, e):
    memory_selected.append({"label": label, "criterion": criterion, "id": str(key), "start": s, "end": e})
    used_keys.add(str(key))

# # ---- 4.1 P1/P2: alinear por guía (SIN fallback) pero "skip si no está en este GOLD"
skipped = []
orphan_examples = []  # ejemplos de guía que NO están en gold/unannotated; solo sirven para few-shot  # para reporte final

def add_from_quote_or_skip(main: str, subcode: str, info: dict):
    label = P_TO_LABEL[main]
    title = (info.get("title") or "").strip()
    crit = (info.get("criterion") or "").strip()

    # tomamos 1er ejemplo entre corchetes
    examples = info.get("examples") or []
    if not examples:
        raise ValueError(f"[SIN FALLBACK] {subcode} no tiene examples en la guía")

    quote = examples[0]
    found = find_quote_in_gold_robust(quote)

    if not found:
        print(f"[WARN] {subcode}: quote no encontrado en GOLD actual -> se marca como HUÉRFANO (solo few-shot).")
        skipped.append({"subcode": subcode, "reason": "quote_not_in_gold", "quote": quote[:200]})
        orphan_examples.append({"subcode": subcode, "label": label, "criterion": f"{subcode} {title} — {crit}".strip(" —"), "quote": quote})
        return

    key, qs, qe = found
    span = pick_annotated_span_covering(key, label, qs, qe)
    if not span:
        raise ValueError(
            f"[SIN FALLBACK] {subcode}: quote encontrado en doc {str(key)[:12]}... "
            f"pero no cae dentro de ningún span anotado de {label}"
        )

    criterion = f"{subcode} {title} — {crit}".strip(" —")
    add_example(label, criterion, key, span[0], span[1])

for main in ["P1", "P2"]:
    # 1) MAIN pseudo si existe
    main_pseudo = f"{main}.MAIN"
    if main_pseudo in guide[main]["subtypes"]:
        add_from_quote_or_skip(main, main_pseudo, guide[main]["subtypes"][main_pseudo])

    # 2) subtipos reales (P1.1, P1.2..., P2.1...)
    for subcode in sorted(guide[main]["subtypes"].keys()):
        if subcode.endswith(".MAIN"):
            continue
        add_from_quote_or_skip(main, subcode, guide[main]["subtypes"][subcode])

print("\n[RESUMEN] subtipos omitidos (no presentes en GOLD actual):", len(skipped))
for s in skipped[:10]:
    print("-", s["subcode"], "|", s["reason"], "|", s["quote"])
if len(skipped) > 10:
    print("... (+", len(skipped) - 10, "más)")


    # Subtipos reales
    for subcode, info in sorted(guide[main]["subtypes"].items()):
        if subcode.endswith(".MAIN"):
            continue
        examples = info.get("examples", [])
        if not examples:
            raise ValueError(f"[SIN FALLBACK] El subtipo {subcode} no tiene ningún ejemplo entre [ ... ] en la guía.")
        quote = examples[0]
        found = find_quote_in_gold_robust(quote)
        if not found:
            raise ValueError(f"[SIN FALLBACK] No encuentro el quote del subtipo {subcode} en GOLD.\nQuote: {quote[:200]}")
        key, qs, qe = found
        span = pick_annotated_span_covering(key, label, qs, qe)
        if not span:
            raise ValueError(f"[SIN FALLBACK] Quote encontrado pero no cae en span anotado {label} para {subcode}")

        title = (info.get("title") or "").strip()
        crit  = (info.get("criterion") or "").strip()
        criterion = f"{subcode} {title} — {crit}".strip(" —")
        add_example(label, criterion, key, span[0], span[1])

# 4.2 P4/P9: variedad formal (4 + 4) sin subtipos (criterio fijo)
def pick_varied_examples(label: str, k: int, patterns: list):
    candidates = pool[label].copy()
    random.shuffle(candidates)
    picked = []
    used_uids = set()

    # primero cubrimos buckets por patrón
    for _, rx in patterns:
        if len(picked) >= k:
            break
        for ex in candidates:
            if ex["id"] in used_uids:
                continue
            if rx.search(ex["text"]):
                picked.append(ex)
                used_uids.add(ex["id"])
                break

    # completamos si faltan (sin repetir uid)
    for ex in candidates:
        if len(picked) >= k:
            break
        if ex["id"] in used_uids:
            continue
        picked.append(ex)
        used_uids.add(ex["id"])

    if len(picked) < k:
        raise ValueError(f"[SIN FALLBACK] No he podido seleccionar {k} ejemplos variados para {label}. Solo {len(picked)}.")
    return picked

dur_patterns = [
    ("formalizacion", re.compile(r"\bformalizaci[oó]n\b", re.IGNORECASE)),
    ("fechas", re.compile(r"\b\d{1,2}\s+de\s+[a-záéíóú]+\s+de\s+\d{4}\b", re.IGNORECASE)),
    ("anos", re.compile(r"\b\d+\s+a[nñ]os?\b", re.IGNORECASE)),
    ("meses", re.compile(r"\b\d+\s+mes(?:es)?\b", re.IGNORECASE)),
    ("dias", re.compile(r"\b\d+\s+d[ií]as?\b", re.IGNORECASE)),
    ("prorroga", re.compile(r"\bpr[oó]rroga\b", re.IGNORECASE)),
]
res_patterns = [
    ("lcsp", re.compile(r"\bLCSP\b|\bLey\s+9/2017\b|\bart[íi]culo(?:s)?\b", re.IGNORECASE)),
    ("causas", re.compile(r"\bcausas?\s+de\s+resoluci[oó]n\b", re.IGNORECASE)),
    ("incumplimiento", re.compile(r"\bincumplim", re.IGNORECASE)),
    ("rescision", re.compile(r"\brescisi[oó]n\b|\brescind", re.IGNORECASE)),
    ("extincion", re.compile(r"\bextinci[oó]n\b", re.IGNORECASE)),
    ("mutuo", re.compile(r"\bmutuo\s+acuerdo\b", re.IGNORECASE)),
]

for ex in pick_varied_examples(P_TO_LABEL["P4"], k=4, patterns=dur_patterns):
    add_example(P_TO_LABEL["P4"], FIXED_CRITERION["P4"], ex["id"], ex["start"], ex["end"])

for ex in pick_varied_examples(P_TO_LABEL["P9"], k=4, patterns=res_patterns):
    add_example(P_TO_LABEL["P9"], FIXED_CRITERION["P9"], ex["id"], ex["start"], ex["end"])

# =========================
# 5) Validación + guardado + bloqueo
# =========================
errors = 0
for ex in memory_selected:
    uid = ex["id"]
    txt = key_to_text.get(uid)
    if not txt:
        errors += 1
        continue
    s, e = ex["start"], ex["end"]
    if not (0 <= s < e <= len(txt)):
        errors += 1
        continue
    if not txt[s:e].strip():
        errors += 1

print("\nmemory_selected tamaño:", len(memory_selected))
print("Docs únicos usados:", len(used_keys))
print("Errores de validación:", errors)
if errors != 0:
    raise ValueError("Hay errores de validación; no debería ocurrir en modo curado.")

# Resumen por etiqueta
cnt = defaultdict(int)
for ex in memory_selected:
    cnt[ex["label"]] += 1
print("Resumen por etiqueta:", dict(cnt))

OUT_CUR = OUT_DIR / "memory_selected_CURATED.json"
OUT_CUR.write_text(json.dumps(memory_selected, ensure_ascii=False, indent=2), encoding="utf-8")
print("Guardado:", OUT_CUR)

blocked = sorted(list(used_keys))
OUT_BLOCK = OUT_DIR / "blocked_doc_uids_by_memory.json"
OUT_BLOCK.write_text(json.dumps(blocked, ensure_ascii=False, indent=2), encoding="utf-8")
print("Guardado:", OUT_BLOCK)



# Guardar ejemplos HUÉRFANOS (solo few-shot; NO bloqueables por doc)
PATH_ORPHAN = OUT_DIR / "guidelines_orphan_examples.json"
PATH_ORPHAN.write_text(json.dumps(orphan_examples, ensure_ascii=False, indent=2), encoding="utf-8")
print("Guardado huérfanos:", PATH_ORPHAN, "| n:", len(orphan_examples))


Tipos detectados: ['P1', 'P2', 'P4', 'P9']
P1 subtipos: 3 | +MAIN_pseudo: 1 | ejemplo: ['P1.1', 'P1.2', 'P1.3']
P2 subtipos: 3 | +MAIN_pseudo: 1 | ejemplo: ['P2.1', 'P2.2', 'P2.3']
P4 subtipos: 9 | +MAIN_pseudo: 0 | ejemplo: ['P4.01', 'P4.02', 'P4.03', 'P4.04', 'P4.05', 'P4.06', 'P4.07', 'P4.08', 'P4.09']
P9 subtipos: 12 | +MAIN_pseudo: 0 | ejemplo: ['P9.01', 'P9.02', 'P9.03', 'P9.04', 'P9.05', 'P9.06', 'P9.07', 'P9.08', 'P9.09', 'P9.10', 'P9.11', 'P9.12']
Docs gold: 373
Spans por etiqueta en gold (pool):
 - DURACION_TOTAL_DEL_CONTRATO : 468
 - OBJETO : 925
 - PRECIO_DEL_CONTRATO : 436
 - RESOLUCION : 526
[WARN] P1.MAIN: quote no encontrado en GOLD actual -> se marca como HUÉRFANO (solo few-shot).
[WARN] P1.1: quote no encontrado en GOLD actual -> se marca como HUÉRFANO (solo few-shot).
[WARN] P1.3: quote no encontrado en GOLD actual -> se marca como HUÉRFANO (solo few-shot).
[WARN] P2.MAIN: quote no encontrado en GOLD actual -> se marca como HUÉRFANO (solo few-shot).
[WARN] P2.1: quot

**Habíamos establecido que la memoria sería más grande. Después de ver [RESUMEN] subtipos omitidos (no presentes en GOLD actual): 6 , nos quedamos con:**
- P1.2
- P2.3
- P4 (4)
- P9 (4)

Debemos ampliar los de P1 y P2 que se han omitido por no aparecer en el gold, para que no se quede cojo.

Los subtipos que omitimos fueron: P1.MAIN, P1.1, P1.3 y P2.MAIN, P2.1, P2.2. Como el objetivo metodológico es “sin fallbacks” + alineado a guía cuando sea posible, lo correcto es:
- **Ampliar memoria hasta 14–16 como acordamos.**
- **Regenerar blocked_doc_uids... a partir de esa memoria final.**

El problema práctico es que varios quotes no aparecen exactamente en el GOLD (ya lo vimos con P1.MAIN y P2.1), así que “sin fallbacks” + “alineación literal exacta” no puede completarse al 100%. La salida más limpia (mínimo cambio y metodológicamente defendible) es:
- P1/P2: intentar alineación literal robusta; si no se encuentra, seleccionar desde gold un span que cumpla el criterio usando heurísticas (no aleatorio puro), y marcarlo como (gold, criterio-sin-cita).
- P4/P9: mantener 4+4 por variedad formal (ya lo tenemos perfecto).
- Esto te da 14–16 sin introducir “fallback aleatorio”, sino “fallback guiado por criterio”.

A continuación, dos celdas:
1. Celda A: ampliar **memory_selected_CURATED.json** que actualmente tiene 10 → 16 con selección guiada para P1/P2 (sin citas).
2. Celda B: regenerar **blocked_doc_uids_by_memory.json** desde la memoria final.

In [23]:
#Celda A — Completar memoria curada a 16 (P1/P2 guiado por criterio, P4/P9 intacto)
from pathlib import Path
import json, re, hashlib, random
from collections import defaultdict

ROOT = Path("/home/jovyan/inesagent")
MEM_DIR = ROOT / "outputs" / "memory"
PATH_CUR = MEM_DIR / "memory_selected_CURATED.json"
assert PATH_CUR.exists(), f"No existe {PATH_CUR}"

P_TO_LABEL = {
    "P1": "OBJETO",
    "P2": "PRECIO_DEL_CONTRATO",
    "P4": "DURACION_TOTAL_DEL_CONTRATO",
    "P9": "RESOLUCION",
}

# --- Subtipos que queremos cubrir para MVP ---
TARGET_P1 = ["P1.MAIN", "P1.1", "P1.2", "P1.3"]   # P1.2 ya lo tienes
TARGET_P2 = ["P2.MAIN", "P2.1", "P2.2", "P2.3"]   # P2.3 ya lo tienes

# --- Heurísticas mínimas (criterio -> regex) para elegir spans desde GOLD ---
# Nota: aplicamos sobre el texto del span anotado (no el doc completo)
RX = {
  # P1
  "P1.MAIN": re.compile(r"\b(objeto|prestaci[oó]n principal|contrato tiene por objeto|finalidad)\b", re.I),
  "P1.1":    re.compile(r"\bexpediente\b|\b\d{4}/[A-Z]{1,6}\d{2}/\d{6,}\b|\b\d{4}/\d+\b", re.I),
  "P1.2":    re.compile(r"\blote\b|\(lote\s*\d+\)|lote\s*\d+", re.I),
  "P1.3":    re.compile(r"[“\"«].{6,}[”\"»]", re.I),  # comillas con contenido (aprox.)

  # P2
  "P2.MAIN": re.compile(r"€|euros?|importe|precio( total)?", re.I),
  "P2.1":    re.compile(r"\bIVA\b|\bIGIC\b|inclu[yi]d[oa]\b|con IVA|IVA incluido", re.I),
  "P2.2":    re.compile(r"\bm[aá]ximo\b|\bl[ií]mite\s+m[aá]ximo\b|\bhasta\s+un\s+importe\b", re.I),
  "P2.3":    re.compile(r"\bp[oó]liza\b|\banualidad(?:es)?\b|\bmensual\b|\brenta\b|\bprecio unitario\b|\bpor persona\b", re.I),
}

def load_json(p: Path):
    return json.loads(p.read_text(encoding="utf-8"))

# --- indexar gold: key_to_text, key_to_tags y pool por etiqueta ---
# Si ya lo tienes hecho arriba en el NB, puedes saltarte este bloque, pero no molesta.
def stable_uid(text: str) -> str:
    return hashlib.sha1(text.encode("utf-8")).hexdigest()

# gold debe existir en tu NB. Si no, carga desde ROOT/gold/corpus_annotated.jsonl
assert "gold" in globals(), "No existe `gold` en memoria. Ejecuta la celda de carga de GOLD antes."

key_to_text = {}
key_to_tags = {}
for d in gold:
    txt = d.get("text","")
    if not txt:
        continue
    uid = stable_uid(txt)
    key_to_text[uid] = txt
    key_to_tags[uid] = d.get("tags", [])

pool = defaultdict(list)
for uid, txt in key_to_text.items():
    for t in key_to_tags.get(uid, []):
        lab = t.get("tag")
        if lab not in set(P_TO_LABEL.values()):
            continue
        s = int(t["start"]); e = int(t["end"])
        if 0 <= s < e <= len(txt):
            span_txt = txt[s:e]
            if span_txt.strip():
                pool[lab].append({"doc_uid": uid, "start": s, "end": e, "text": span_txt})

# --- cargar memoria actual ---
mem = load_json(PATH_CUR)

# helper: qué criterios ya están cubiertos
def covered_codes(mem_list):
    codes = set()
    for x in mem_list:
        crit = (x.get("criterion","") or "")
        # tomamos el primer token "P?.?" o "P?.MAIN" si aparece al inicio
        m = re.match(r"^\s*(P(?:1|2)\.(?:MAIN|\d+))\b", crit)
        if m:
            codes.add(m.group(1))
    return codes

covered = covered_codes(mem)

def add_from_pool(label: str, code: str):
    """
    Elige un span anotado que matchee el regex del subtipo (si puede),
    evitando repetir doc_uid ya usados en memoria.
    """
    used_uids = {x["doc_uid"] for x in mem}
    candidates = pool[label]
    if not candidates:
        raise ValueError(f"No hay pool para {label}")

    rx = RX.get(code)
    # 1) preferir match regex
    if rx:
        for ex in candidates:
            if ex["doc_uid"] in used_uids:
                continue
            if rx.search(ex["text"]):
                return ex

    # 2) si no hay match, elegimos uno "no repetido" con longitud razonable (evita spans 0)
    candidates2 = [ex for ex in candidates if ex["doc_uid"] not in used_uids and len(ex["text"]) >= 20]
    if candidates2:
        # elige el más “informativo” (más largo) para estabilizar
        return sorted(candidates2, key=lambda z: len(z["text"]), reverse=True)[0]

    # 3) último recurso: permitir doc_uid repetido (pero esto normalmente no pasa)
    return sorted(candidates, key=lambda z: len(z["text"]), reverse=True)[0]

def push_example(label: str, criterion: str, ex: dict):
    mem.append({
        "label": label,
        "criterion": criterion,
        "doc_uid": ex["doc_uid"],
        "start": ex["start"],
        "end": ex["end"],
    })

# --- completar P1/P2 hasta cubrir TARGETS ---
def criterion_text(code: str):
    # texto corto y explícito: método = gold + heurística
    # (sin citar guía porque el literal no está en gold)
    mapping = {
        "P1.MAIN": "P1.MAIN — criterio general (gold; selección heurística)",
        "P1.1":    "P1.1 Inclusión de expedientes en objeto — (gold; selección heurística)",
        "P1.2":    "P1.2 Inclusión del lote en el objeto — (gold)",
        "P1.3":    "P1.3 Delimitación del objeto entrecomillado — (gold; selección heurística)",
        "P2.MAIN": "P2.MAIN — criterio general (gold; selección heurística)",
        "P2.1":    "P2.1 Importes con IVA/IGIC — (gold; selección heurística)",
        "P2.2":    "P2.2 Importes máximos — (gold; selección heurística)",
        "P2.3":    "P2.3 Precios en otros formatos — (gold)",
    }
    return mapping.get(code, f"{code} — (gold; selección heurística)")

# P1
for code in TARGET_P1:
    if code in covered:
        continue
    ex = add_from_pool(P_TO_LABEL["P1"], code)
    push_example(P_TO_LABEL["P1"], criterion_text(code), ex)
    covered.add(code)

# P2
for code in TARGET_P2:
    if code in covered:
        continue
    ex = add_from_pool(P_TO_LABEL["P2"], code)
    push_example(P_TO_LABEL["P2"], criterion_text(code), ex)
    covered.add(code)

# --- sanity: esperamos 16 (4 P1 + 4 P2 + 4 P4 + 4 P9) ---
print("Memory size AFTER:", len(mem))
counts = defaultdict(int)
for x in mem:
    counts[x["label"]] += 1
print("Counts by label:", dict(counts))

# validar offsets
errors = 0
for x in mem:
    uid = x["doc_uid"]
    txt = key_to_text.get(uid, "")
    s, e = int(x["start"]), int(x["end"])
    if not txt or not (0 <= s < e <= len(txt)) or not txt[s:e].strip():
        errors += 1
print("Offset errors:", errors)

# Guardar como UPDATED curated
PATH_OUT = MEM_DIR / "memory_selected_CURATED.json"
PATH_OUT.write_text(json.dumps(mem, ensure_ascii=False, indent=2), encoding="utf-8")
print("✅ Guardado:", PATH_OUT)


KeyError: 'doc_uid'

**Con esto conseguimos:**
- Mantener los 10 items.
- Añadir los 6 que faltan para completar:
    - P1.MAIN, P1.1, P1.3
    - P2.MAIN, P2.1, P2.2

Sin aleatoriedad pura: intenta un span que case con un regex razonable del subtipo y evita repetir doc_uid.

In [ ]:
#Celda B — Regenerar blocked_keys_by_memory.json desde la memoria final 
from pathlib import Path
import json

ROOT = Path("/home/jovyan/inesagent")
MEM_DIR = ROOT / "outputs" / "memory"

PATH_MEM = MEM_DIR / "memory_selected_CURATED.json"
assert PATH_MEM.exists(), f"No existe {PATH_MEM}"

mem = json.loads(PATH_MEM.read_text(encoding="utf-8"))
blocked_keys = sorted({x["doc_uid"] for x in mem})

# Intentamos inferir ids vs uids: si son int -> id; si son str, lo tratamos como id si existe en gold_id_set
# (si esta celda se ejecuta sin cargar gold_id_set, caerá a uids)
try:
    gold_id_set
except NameError:
    gold_id_set = set()

blocked_ids = sorted({k for k in blocked_keys if isinstance(k, int) or (isinstance(k,str) and k in gold_id_set)})
blocked_uids_only = sorted({k for k in blocked_keys if k not in set(blocked_ids)})

OUT = MEM_DIR / "blocked_keys_by_memory.json"
OUT.write_text(json.dumps({"blocked_ids": blocked_ids, "blocked_uids": blocked_uids_only}, ensure_ascii=False, indent=2), encoding="utf-8")

# legacy
OUT_LEG = MEM_DIR / "blocked_doc_uids_by_memory.json"
OUT_LEG.write_text(json.dumps(blocked_keys, ensure_ascii=False, indent=2), encoding="utf-8")

print("✅ blocked_keys_by_memory.json actualizado")
print("ids:", len(blocked_ids), "| uids:", len(blocked_uids_only))
print("legacy:", OUT_LEG)


Una vez tenemos **memory_selected_CURATED.json**
1. Creamos el fichero de bloqueo real
2. Lo usamos en NB02 para excluir esos docs de val/test/prompt_regression

- Este **blocked_doc_uids_by_memory.json** se va consumir en NB02 para excluir docs de val/test/prompt_regression.
- Guardamos también la memoria final que vas a usar en prompting
- Si vamos a usar memory_selected_CURATED.json, nos aseguramos de que memory_selected apunta a esa versión y la guardamos como “final” (para no liarnos)

In [ ]:
#para que no haya confusion, podemos añadir "FINAL" a los documentos que se van a estar usando en los otros NB

**En NB03 terminamos con memoria + bloqueo listos. El siguiente paso es volver al notebook NB02 (splits), donde lo reharemos usando el fichero de bloqueo.**

Objetivo de NB02 ahora:
- Generar train/val/test/prompt_regression
- asegurando que NINGÚN doc_uid en blocked_doc_uids_by_memory.json aparezca en:
    - val
    - test
    - prompt_regression

(En train podemos decidir incluirlos o no; para MVP, lo habitual es permitirlos en train pero nunca en evaluación.)